In [ ]:
import gurobipy as gp
from gurobipy import GRB
import numpy as np
from scipy.special import comb
import matplotlib.pyplot as plt

# Set up parameters
n = 10  # Degree of Bézier curve
M = 20  # Number of test points

# Define matrix A
A = np.array([[2, 3],
              [4, 3]])

# Create Gurobi model
model = gp.Model("ODE_system_solver")

# Add variables (control points for both y1 and y2)
p1 = model.addVars(n+1, lb=-GRB.INFINITY, ub=GRB.INFINITY, name="p1")
p2 = model.addVars(n+1, lb=-GRB.INFINITY, ub=GRB.INFINITY, name="p2")

# Add constraints for initial conditions
model.addConstr(p1[0] == 1, "initial_condition_y1")
model.addConstr(p2[0] == 1, "initial_condition_y2")

# Define test points
test_points = np.linspace(0, 1, M)

# Define Bernstein polynomial function
def bernstein(i, n, t):
    return comb(n, i, exact=True) * (t**i) * ((1-t)**(n-i))

# Set up objective function
obj = 0
for t in test_points:
    y1 = sum(p1[i] * bernstein(i, n, t) for i in range(n+1))
    y2 = sum(p2[i] * bernstein(i, n, t) for i in range(n+1))
    y1_prime = n * sum((p1[i+1] - p1[i]) * bernstein(i, n-1, t) for i in range(n))
    y2_prime = n * sum((p2[i+1] - p2[i]) * bernstein(i, n-1, t) for i in range(n))
    
    g1 = y1_prime - (A[0,0]*y1 + A[0,1]*y2)
    g2 = y2_prime - (A[1,0]*y1 + A[1,1]*y2)
    
    obj += g1*g1 + g2*g2

model.setObjective(obj, GRB.MINIMIZE)

# Optimize the model
model.optimize()

# Extract the solution
control_points1 = [p1[i].X for i in range(n+1)]
control_points2 = [p2[i].X for i in range(n+1)]

# Define function to evaluate Bézier curve
def bezier_curve(t, control_points):
    n = len(control_points) - 1
    return sum(control_points[i] * bernstein(i, n, t) for i in range(n+1))

# Plot the results
t_plot = np.linspace(0, 1, 100)
y1_plot = [bezier_curve(t, control_points1) for t in t_plot]
y2_plot = [bezier_curve(t, control_points2) for t in t_plot]

# Analytical solution
def analytical_solution(t):
    lambda1, lambda2 = np.linalg.eigvals(A)
    v1, v2 = np.linalg.eig(A)[1].T
    c1 = (7*v2[1] - 7*v2[0]) / (7*(v1[0]*v2[1] - v1[1]*v2[0]))
    c2 = (7*v1[0] - 7*v1[1]) / (7*(v1[0]*v2[1] - v1[1]*v2[0]))
    return c1 * v1 * np.exp(lambda1*t) + c2 * v2 * np.exp(lambda2*t)

y_analytical = np.array([analytical_solution(t) for t in t_plot])

plt.figure(figsize=(10, 6))
plt.plot(t_plot, y1_plot, label='y1 Bézier curve')
plt.plot(t_plot, y2_plot, label='y2 Bézier curve')
plt.plot(t_plot, y_analytical[:,0], '--', label='y1 Analytical')
plt.plot(t_plot, y_analytical[:,1], '--', label='y2 Analytical')
plt.legend()
plt.xlabel('t')
plt.ylabel('y(t)')
plt.title('Solution of y\'(t) = Ay(t), y(0) = (1, 1)^T')
plt.grid(True)
plt.show()

print("Control points for y1:", control_points1)
print("Control points for y2:", control_points2)

Restricted license - for non-production use only - expires 2025-11-24
Gurobi Optimizer version 11.0.2 build v11.0.2rc0 (mac64[x86] - Darwin 22.6.0 22G74)

CPU model: Intel(R) Core(TM) i7-8750H CPU @ 2.20GHz
Thread count: 6 physical cores, 12 logical processors, using up to 12 threads

Optimize a model with 2 rows, 22 columns and 2 nonzeros
Model fingerprint: 0x2daee8e0
Model has 253 quadratic objective terms
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [0e+00, 0e+00]
  QObjective range [6e-05, 6e+02]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 1e+00]
Presolve removed 2 rows and 2 columns
Presolve time: 0.01s
Presolved: 0 rows, 20 columns, 0 nonzeros
Presolved model has 210 quadratic objective terms
Ordering time: 0.00s

Barrier statistics:
 Free vars  : 39
 AA' NZ     : 1.710e+02
 Factor NZ  : 1.900e+02
...

Barrier solved model in 20 iterations and 0.04 seconds (0.00 work units)
Optimal objective 3.53920757e-04

Output is truncated. View as a scrollable element or open in a text editor. Adjust cell output settings...

Control points for y1: [1.0, 1.4999500397437784, 2.353275609485717, 3.747822899255382, 6.442296825633735, 10.583243981932005, 19.610787692466143, 34.722762950852385, 69.22566628372547, 138.40417463210292, 345.91824640016705]
Control points for y2: [1.0, 1.6999281058319156, 2.867698937509111, 4.751582788902434, 8.372659558072638, 13.907190136939215, 25.970779715365605, 46.13156549840758, 92.15283153191676, 184.4039933363164, 461.10169403887505]